In [1]:
import numpy as np
import pandas as pd
import json
import requests 
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


from bs4 import BeautifulSoup
#from pandas.io.json import json_normalize
#from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim
#!pip install folium
#import folium

print('Done!')

Done!


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page=requests.get(url,'lxml').text
soup=BeautifulSoup(wiki_page,'lxml')

header=['Postcode', 'Borough', 'Neighbourhood']
core=[]

tabl=soup.find('table')
for tr in tabl.find_all('tr'):
    td=tr.find_all('td')
    if len(td)==3:
      instance=['','','']
      instance[0]=td[0].text.strip()
      instance[1]=td[1].text.strip()
      if td[2].text.strip().replace('\n','')=='Not assigned':
           instance[2]=td[1].text.strip()
      else:
           instance[2]=td[2].text.strip().replace('\n','') 
      
      core.append(instance)
    
df=pd.DataFrame.from_records(core, columns=header)

df1=df[df.Borough != 'Not assigned']
df1.sort_values(by=['Postcode'])
df1.reset_index(drop=True, inplace=True)
df1.head(200)



,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [3]:
df2=df1.copy()
i=0
df2.is_copy=False
for i in range(df2.shape[0]):
    if i != df2.shape[0]-1:
        if df2.loc[i,'Postcode']==df2.loc[i+1,'Postcode']:
            df2.loc[i+1,'Borough']='Duplicate'
            df2.loc[i,'Neighbourhood']=df2.loc[i,'Neighbourhood']+', '+df2.loc[i+1,'Neighbourhood']
            for e in range(i+1,df2.shape[0]):
                if e != df2.shape[0]-1: 
                    if df2.loc[e,'Postcode']==df2.loc[e+1,'Postcode']:
                        df2.loc[e+1,'Borough']='Duplicate'
                        df2.loc[i,'Neighbourhood']=df2.loc[i,'Neighbourhood']+', '+df2.loc[e+1,'Neighbourhood']
                    else:
                        i=e+1
                        break
df3=df2[df2.Borough != 'Duplicate']                     
df3.head(100)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

NameError: name 'df_ab' is not defined